In [1]:
from tqdm.notebook import tqdm
import pandas as pd 
from time import sleep

import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader
from torchvision import  transforms


In [2]:
# activate GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# load dataset in mini-batch format by DataLoader

In [3]:
class MNIST(Dataset):
    def __init__(self, src_path, transform=None):
        self.transform = transform
        df = pd.read_csv(src_path)
        y_np = df.iloc[:, 0].values
        X_np = df.iloc[:, 1:].values/255.0

        self.y = torch.from_numpy(y_np)
        X = torch.from_numpy(X_np)
        self.X = torch.reshape(X,(-1, 1, 28 ,28)).float()
    
    def __len__(self):
        assert len(self.y) == self.X.shape[0]
        return len(self.y)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        X_sub = self.X[idx, :]
        y_sub = self.y[idx]

        if self.transform:
            X_sub = self.transform(X_sub)

        return X_sub, y_sub



In [4]:

transformer = transforms.Compose([
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

dataset = MNIST('../../data/datasets/train.csv')
batch_size = 20
loader = DataLoader(dataset,batch_size=batch_size, shuffle=True, num_workers=0)



# build ConvNet architecture

In [5]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Linear(in_features=32*5*5, out_features=128)
        self.fc1 = nn.Linear(in_features=128, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)
    
    def forward(self, x):
        in_size = x.size(0)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*5*5)
        x = F.relu(self.flatten(x))
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))     ## this is the output layer.
        #x = F.log_softmax(x)        ## this is the logits layer.
        x = self.fc2(x)
        return x 

model = ConvNet()
model = model.to(device)


# define loss function, matrics and optimization algorithm.

In [6]:

criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()

optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


 train the model

In [7]:

epochs = 1
for epoch in range(epochs):
        for i, data in enumerate(loader):
            inputs, labels = data 
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if i % 100 == 0:
                print(f'[{epoch+1} : {i+1}] loss : {loss.item()}')
print('Finished Training')


[1 : 1] loss : 2.285472869873047
[1 : 101] loss : 2.307098865509033
[1 : 201] loss : 2.30499529838562
[1 : 301] loss : 2.2906596660614014
[1 : 401] loss : 2.281426191329956
[1 : 501] loss : 2.310518264770508
[1 : 601] loss : 2.283036470413208
[1 : 701] loss : 2.301621437072754
[1 : 801] loss : 2.328727960586548
[1 : 901] loss : 2.2873597145080566
[1 : 1001] loss : 2.2834925651550293
[1 : 1101] loss : 2.3116073608398438
[1 : 1201] loss : 2.310828685760498
[1 : 1301] loss : 2.3072009086608887
[1 : 1401] loss : 2.3113882541656494
[1 : 1501] loss : 2.322164297103882
[1 : 1601] loss : 2.2963826656341553
[1 : 1701] loss : 2.306656837463379
[1 : 1801] loss : 2.283966302871704
[1 : 1901] loss : 2.2896173000335693
[1 : 2001] loss : 2.3055148124694824
Finished Training


In [8]:

epochs = 1
for epoch in range(epochs):
    with tqdm(loader, unit='batch', position=0, leave=True) as progress_bar:
        for data in progress_bar:
            progress_bar.set_description(f'Epoch {epoch}:')

            inputs, labels = data 
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            correct = (torch.argmax(outputs, axis=1) == labels).sum().item()
            accuracy = correct / batch_size
            progress_bar.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
            #progress_bar.update()

print('Finished Training')


  0%|          | 0/2100 [00:00<?, ?batch/s]

Finished Training


# save model results

In [11]:

torch.save(model.state_dict(), '../../data/models/convNet_torch.pt')

In [12]:
torch.save(loader, '../../data/models/dataloader.pth')